In [1]:
import sys
sys.path.append('../')
import os
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from tqdm.notebook import trange
from instances.instance import *
from GeCo_Instance_generators.knapsack import  yang_instance
from GeCo_Instance_generators.packing import tang_instance_packing

In [2]:
generator = StochasticBinPackerGenerator()
problem = generator.generate_problem(10,10,20,20,30, "multipeak")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-20


In [3]:
MP = gp.Model("MP")
x = MP.addMVar((problem.s1_n_var,), name = "x")
#y = EP.addMVar((problem.s2_n_var, problem.k), name = "y")
MP.modelSense = problem.s1_direction
if problem.s2_direction == problem.s1_direction:
    MP.setObjective(
        problem.c @ x
    )
else:
    MP.setObjective(
        problem.c @ x
    )

MP.addConstr(
    problem.A @ x <= problem.b
)
MP.optimize()
mp_x = x.x

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 10 columns and 154 nonzeros
Model fingerprint: 0xc491e529
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 1e+02]
Presolve time: 0.01s
Presolved: 20 rows, 10 columns, 154 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+31   9.775000e+31   1.800000e+01      0s
       6    2.3394631e+02   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.339463087e+02


In [4]:
SP = gp.Model("SP")
A = problem.other_info['original_A2']
b = problem.other_info['original_b2']
c = problem.other_info['original_c2']

y = SP.addMVar((A.shape[1], ), obj = c)
SP.modelSense = problem.s2_direction

SP.addConstr(
    A @ y <= b
)
SP.optimize()
y.x

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 10 columns and 151 nonzeros
Model fingerprint: 0x0855414e
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 1e+02]
Presolve time: 0.00s
Presolved: 20 rows, 10 columns, 151 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.6250000e+30   8.987500e+31   8.625000e+00      0s
       9    1.1387500e+02   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.138750000e+02


array([ 1.25,  2.  ,  0.  ,  0.  ,  0.  , 20.  ,  2.  ,  4.5 ,  0.  ,
        0.  ])

In [5]:
EP = gp.Model("EP")
x = EP.addMVar((problem.s1_n_var,), name = "x")
y = EP.addMVar((problem.s2_n_var, problem.k), name = "y")
EP.modelSense = problem.s1_direction
if problem.s2_direction == problem.s1_direction:
    EP.setObjective(
        problem.c @ x + gp.quicksum(1 / problem.k * problem.q_list[s] @ y[:,s] for s in range(problem.k))
    )
else:
    EP.setObjective(
        problem.c @ x - gp.quicksum(1 / problem.k * problem.q_list[s] @ y[:,s] for s in range(problem.k))
    )

EP.addConstr(
    problem.A @ x <= problem.b
)

EP.addConstrs((
    problem.W_list[s] @ y[:,s] + problem.T_list[s] @ x <= problem.h_list[s] for s in range(problem.k)
))

EP.optimize()
ep_x = x.x

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 620 rows, 310 columns and 10611 nonzeros
Model fingerprint: 0xbef6e495
Coefficient statistics:
  Matrix range     [2e-02, 4e+01]
  Objective range  [2e-02, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 2e+02]
Presolve time: 0.00s
Presolved: 620 rows, 310 columns, 10611 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7466667e+31   7.902794e+32   2.746667e+01      0s
     281    9.6996816e+02   0.000000e+00   0.000000e+00      0s

Solved in 281 iterations and 0.02 seconds (0.01 work units)
Optimal objective  9.699681619e+02


problems are defined by: 
c, A, B (first stage jazz)

q_s, W_s, h_s, T_s, and clustering variables

In [6]:
MP = gp.Model("MP")
MP.Params.outputFlag = 0
x = MP.addMVar((problem.s1_n_var,), name = "x")
eta = MP.addMVar((problem.k,), name = "eta", ub = problem.eta_bounds[1], lb = problem.eta_bounds[0])

if problem.s1_direction == GRB.MAXIMIZE:
    MP.modelSense = GRB.MAXIMIZE
else:
    MP.modelSense = GRB.MINIMIZE

MP.setObjective(
    problem.c @ x + np.array([1/problem.k] * problem.k) @ eta 
)

c1 = MP.addConstr(
    problem.A @ x <= problem.b
)


cut_found = True
n_iters = 0
try: 
    while cut_found:
        cut_found = False
        q_vals = []
        if n_iters < 200:
            n_iters = n_iters + 1
            print(f"On Iteration: {n_iters}")
        else:
            break
        MP.update()
        MP.optimize()
        LB = 0
        x_i = x.x
        eta_i = eta.x
        for s in range(problem.k):
            SP = gp.Model("SP")
            SP.Params.outputFlag = 0  # turn off output
            SP.Params.method = 1      # dual simplex
            y = SP.addMVar((problem.s2_n_var,), name = "y", obj = problem.q_list[s])
            if problem.s2_direction == GRB.MAXIMIZE:
                SP.modelSense = GRB.MAXIMIZE
            else:
                SP.modelSense = GRB.MINIMIZE
            res = SP.addConstr(
                problem.W_list[s] @ y <= problem.h_list[s] - (problem.T_list[s] @ x_i)
            )
            SP.optimize()
            Q = SP.ObjVal
            q_vals.append(Q)
            pi = res.Pi
            LB = LB + Q / problem.k
            if np.abs(Q - eta_i[s]) > 0.00001:
                if Q < eta_i[s]:
                    cut_found = True
                    p1 = pi @ problem.h_list[s]
                    p2 = pi @ problem.T_list[s]
                    if problem.s2_direction == GRB.MAXIMIZE:
                        MP.addConstr(
                            eta[s] <= p1 - gp.quicksum(p2[a] * x[a] for a in range(problem.s1_n_var))
                        )
                    else:
                        MP.addConstr(
                            eta[s] >= p1 - gp.quicksum(p2[a] * x[a] for a in range(problem.s1_n_var))
                        )        
        print(f"Iteration {n_iters}: LB = {LB}. UB = {MP.ObjVal}")
except:
    print(f"Errored out on iteration: {n_iters} scenario {s}")



On Iteration: 1
Iteration 1: LB = 665.1442974381071. UB = 9233.946308724833
On Iteration: 2
Iteration 2: LB = 824.6840365500047. UB = 1027.7324776524215
On Iteration: 3
Iteration 3: LB = 796.5607848707833. UB = 972.2914734503945
On Iteration: 4
Iteration 4: LB = 806.4683465180764. UB = 970.0137261519591
On Iteration: 5
Iteration 5: LB = 806.0382739197388. UB = 969.9778685445525
On Iteration: 6
Iteration 6: LB = 806.278768244491. UB = 969.9681618651525
